# Describe a PIV Result File

In this example, we want to describe the result file of the ILA Vortex pair (source: https://www.pivtec.com/download/samples/VortexPairSeq.zip)

Where to start? Let's first evaluate what we have:

General context:
- The general concept of a dataset is described by `dcat:Dataset`
- The file is described by `pivmeta:PIVResultDistribution` and is part of the `dcat:Dataset`

Specific information:<br>
Of greater interest is the PIV process including the PIV parameters leading to the dataset. A `dcat:Dataset` is the output of the PIV process. For this, we can use [`m4i:output of`](http://purl.obolibrary.org/obo/RO_0002353).

## Preparation and Imports

In [1]:
from pivmetalib import pivmeta, dcat, m4i, schema, sd, prov
from pivmetalib.namespace import PIVMETA

from ssnolib import StandardNameTable

We will describe variables using standard names. For this we, decide to use [this Standard Name table](https://zenodo.org/records/14175299)

In [2]:
snt = StandardNameTable.download(url="https://zenodo.org/records/14175299/files/Standard_Name_Table_for_Particle_Image_Velociemtry_data.jsonld?download=1", fmt="jsonld")
standard_names = snt.get_standard_names_as_frozen_dataclass()

In [3]:
result_dist = pivmeta.PIVResultDistribution(
    title='Result File',
    downloadURL='file:///vp1a.dat'
)
# the "downloaded" file must exist:
assert result_dist.download().exists()

In [4]:
piv_image_dist = pivmeta.PIVImageDistribution(
    title='ILA Vortex Pair Images',
    download_URL='https://www.pivtec.com/download/samples/pivimg1.zip',
    piv_image_type=PIVMETA.ExperimentalImage
)
piv_image_dist

PIVImageDistribution(id=_:N29b71c4820ed4f9595d0e7c59b9e6601, title=ILA Vortex Pair Images, download_URL=https://www.pivtec.com/download/samples/pivimg1.zip, piv_image_type=https://matthiasprobst.github.io/pivmeta#ExperimentalImage)

In [5]:
ds = dcat.Dataset(
    title='ILA Vortex Pair',
    distribution=[result_dist, piv_image_dist]
)
ds

Dataset(id=_:N271665700cb2492aa2be20245e56a178, title=ILA Vortex Pair, distribution=[PIVResultDistribution(id=_:Ndff5ba8a54994c959c89e52bfeba6f3f, title=Result File, download_URL=file:///vp1a.dat), PIVImageDistribution(id=_:N29b71c4820ed4f9595d0e7c59b9e6601, title=ILA Vortex Pair Images, download_URL=https://www.pivtec.com/download/samples/pivimg1.zip, piv_image_type=https://matthiasprobst.github.io/pivmeta#ExperimentalImage)])

Before defining the PIV processing steps, we need to describe the software used:

In [6]:
piv_software = pivmeta.PIVSoftware(
    author=prov.Organization(
        name='PIVTEC GmbH',
        mbox='info@pivtec.com',
        url='https://www.pivtec.com/'
    ),
    has_documentation='https://www.pivtec.com/download/docs/PIVview_v36_Manual.pdf',
)

In [7]:
# import h5py
# from typing import Dict


# import h5rdmtoolbox as h5tbx
# h5tbx.set_config(natural_naming=False)

# with h5tbx.File() as h5:
#     piv_software.dump_hdf(h5)
#     h5.dump()
#
# print(piv_software.dump_jsonld())

## Processing steps

A `PIVProcessingStep` class is provided in order to distinguish the processing step from others. Some methods are provided as classes but without specific properties. This is done to provide flexibility, as all methods can be standardized. However, by introducing **standard names**, the authors may narrow their parameter definitions either by using global standard names (with an IRI) or within their project.

*TODO: Put here an image illustrating the possibilities*

By using standard names, important parameters can be identified unambiguously. See `PIVMETA.image_filter_kernel_size` in the example in contrast to the 180° image rotation.

### 1. Pre-Processing (Image processing)

Methods:
- image rotation by 180 deg

In [8]:
from pivmetalib import PIVMETA

In [9]:
pre = pivmeta.PIVPreProcessing(
    name='Image pre processing',
    realizesMethod=[
        m4i.Method(
            description='Rotates the input image by 180 deg',
            parameter=m4i.NumericalVariable(
                name='rotation',
                value=180,
                unit='deg',
                quantity_kind='https://qudt.org/vocab/unit/DEG'
            )
        ),
        # Dont define all the classes for filters and outlier detection because everybody may define it differently.
        # common parameters can be specified "on demand" by standard names like so: 
        m4i.Method(
            description='Applies a median filter to the image',
            hasParameter=pivmeta.NumericalVariable(
                standard_name=standard_names.image_filter_kernel_size,
                value=3,
            )
        )
    ]
)
pre

PIVPreProcessing(id=_:N32a709df8a494ea8bf6a355acd7793e2, realizes_method=[Method(id=_:Na2cb19f42f2e4fd6a90529ce055328e8, description=Rotates the input image by 180 deg, parameter=NumericalVariable(id=_:Nf8cf19c8f163478da4103bf2d68243f2, hasNumericalValue=180, hasUnit=http://qudt.org/vocab/unit/DEG, hasKindOfQuantity=https://qudt.org/vocab/unit/DEG, name=rotation)), Method(id=_:N452ea88649b14fa49a7e69f7341f6fc6, description=Applies a median filter to the image, hasParameter=NumericalVariable(id=_:Ne692b2144c9847b2a93cda45c5fc2511, hasNumericalValue=3, hasStandardName=image_filter_kernel_size))], name=Image pre processing)

## 2. PIV evaluation

The PIV evaluation is a `m4i:ProcessingStep`. It realizes the methods
- `pivemta:CorrelationMethod` (e.g. FFT, ... ),
- `pivemta:InterrogationMethod` (e.g. `pivemta:Multigrid`, ...) and
- `pivemta:PeakSearchMethod`.

The outlier detection (validation) is a sub-processing step (`pivemta:PIVValidation`), because it can realize multiple methods. Most popular ones are described by classes:
- `pivemta:DynMean`.

Here, a multigrid evaluation is performed using standard FFT.

### 2.1 Correlation algorithm

The correlation algorithm is a subclass of `m4i:Method`. At least a name and description should be provided (here, taken from the documentation). We could also provide parameters.

In [10]:
calgo = pivmeta.CorrelationMethod(
    name='Standard (FFT) Correlation',
    windowWeightingFunction=PIVMETA.SquareWindow,
    description='The default mode of cross-correlation using FFTs to speed the computation. ' 
    'In principle the sum of pixel-wise multiplication of intensities is computed for each ' 
    'correlation offset (For implementation details please refer to Raffel et al. 2007).'
)
print(calgo.model_dump_jsonld())

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "m4i": "http://w3id.org/nfdi4ing/metadata4ing#",
        "schema": "https://schema.org/",
        "pivmeta": "https://matthiasprobst.github.io/pivmeta#"
    },
    "@type": "pivmeta:CorrelationMethod",
    "schema:description": "The default mode of cross-correlation using FFTs to speed the computation. In principle the sum of pixel-wise multiplication of intensities is computed for each correlation offset (For implementation details please refer to Raffel et al. 2007).",
    "pivmeta:windowWeightingFunction": "https://matthiasprobst.github.io/pivmeta#SquareWindow",
    "name": "Standard (FFT) Correlation",
    "@id": "_:Nc5422f210cdf43cfb7637177240a0f40"
}


### 2.2 Interrogation method

In this example, a *Multi-Grid* method was used starting from a window with size 64 px down to 16 px in 3 steps

In [11]:
v = pivmeta.NumericalVariable(
            standard_name=standard_names.x_initial_interrogation_window_size,
            value=64,
        )
print(v.model_dump_jsonld(resolve_keys=False))

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "m4i": "http://w3id.org/nfdi4ing/metadata4ing#",
        "ssno": "https://matthiasprobst.github.io/ssno#",
        "pivmeta": "https://matthiasprobst.github.io/pivmeta#",
        "skos": "http://www.w3.org/2004/02/skos/core#",
        "dcat": "http://www.w3.org/ns/dcat#"
    },
    "@type": "m4i:NumericalVariable",
    "m4i:hasNumericalValue": 64,
    "pivmeta:hasStandardName": {
        "@type": "ssno:StandardName",
        "ssno:standardName": "x_initial_interrogation_window_size",
        "ssno:unit": "http://qudt.org/vocab/unit/PIXEL",
        "ssno:description": "Width of a initial PIV interrogation window (in case of a multi-grid PIV algorithm).",
        "@id": "https://doi.org/10.5281/zenodo.14175299/N89d0dcc535b64f2f98ada4fc152f4586"
    },
    "@id": "_:Nb87a362d008d42989cce94ecf763dca4"
}


In [12]:
#print(v.dump_jsonld())

In [13]:
int_meth = m4i.Method(
    name='Multigrid interrogation method',
    description='Run a multigrid PIV algorithm on all images',
    hasParameter=[
        pivmeta.NumericalVariable(
            standard_name=standard_names.x_initial_interrogation_window_size,
            value=64,
        ),
        pivmeta.NumericalVariable(
            standard_name=standard_names.y_initial_interrogation_window_size,
            value=64,
        ),
        pivmeta.NumericalVariable(
            standard_name=standard_names.x_final_interrogation_window_size,
            value=16,
        ),
        pivmeta.NumericalVariable(
            standard_name=standard_names.y_final_interrogation_window_size,
            value=16,
        ),
        pivmeta.NumericalVariable(
            standard_name=standard_names.x_final_interrogation_window_overlap_size,
            value=8,
        ),
        pivmeta.NumericalVariable(
            standard_name=standard_names.y_final_interrogation_window_overlap_size,
            value=8,
        ),
        pivmeta.NumericalVariable(
            # hasStandardName=PIVMETA.number_of_multigrid_passes,
            description='Number of multigrid passes',
            value=3,
        )
    ]
)
int_meth.hasParameter[-1]

NumericalVariable(id=_:N38a28dc3d9d94f36ad57aa99d2a579c6, description=Number of multigrid passes, hasNumericalValue=3)

### 2.3 Outlier detection

We use the following two methods for outlier detection:
- normalized median test threshold: 3.0 (see DOI=https://doi.org/10.1007/s00348-005-0016-6)
- dynamic mean test: mean=2.0, var=1.0

In [14]:
median_test = pivmeta.OutlierDetectionMethod(
    name='normalized median test',
    hasParameter=m4i.NumericalVariable(
        label='threshold',
        value=3.0
    )
)

In [15]:
dyn_mean = pivmeta.OutlierDetectionMethod(
    name='dynamic mean test',
    hasParameter=[
        m4i.NumericalVariable(
            label='mean',
            value=2.0
        ),
        m4i.NumericalVariable(
            label='var',
            value=1.0
        )
    ]
)

In [16]:
proc = pivmeta.PIVEvaluation(
    label='piv evaluation',
    realizesMethod=[
        calgo,
        int_meth,
        median_test,
        dyn_mean
    ]
)
proc

PIVEvaluation(id=_:N52794fb4b0ae4be6b81ec2548057f57e, label=piv evaluation, realizes_method=[CorrelationMethod(id=_:Nc5422f210cdf43cfb7637177240a0f40, description=The default mode of cross-correlation using FFTs to speed the computation. In principle the sum of pixel-wise multiplication of intensities is computed for each correlation offset (For implementation details please refer to Raffel et al. 2007)., windowWeightingFunction=https://matthiasprobst.github.io/pivmeta#SquareWindow, name=Standard (FFT) Correlation), Method(id=_:Ne05a4b14d01a496f83f0666f9234d963, description=Run a multigrid PIV algorithm on all images, name=Multigrid interrogation method, hasParameter=[NumericalVariable(id=_:Nbdf2465bff464c69aac1e7f3df8ce748, hasNumericalValue=64, hasStandardName=x_initial_interrogation_window_size), NumericalVariable(id=_:N6d5a2a7f99724b4d9caf89be55621601, hasNumericalValue=64, hasStandardName=y_initial_interrogation_window_size), NumericalVariable(id=_:N4bef225d50584c2495b5b93a8e1f02f5, hasNumericalValue=16, hasStandardName=x_final_interrogation_window_size), NumericalVariable(id=_:N4bbf06cab8b140038d9d4d3fb98052a6, hasNumericalValue=16, hasStandardName=y_final_interrogation_window_size), NumericalVariable(id=_:Na2973b9b366e4d9c9c964bec666f47cd, hasNumericalValue=8, hasStandardName=x_final_interrogation_window_overlap_size), NumericalVariable(id=_:Nacd97eab300246d3bacd06478c1ca23b, hasNumericalValue=8, hasStandardName=y_final_interrogation_window_overlap_size), NumericalVariable(id=_:N38a28dc3d9d94f36ad57aa99d2a579c6, description=Number of multigrid passes, hasNumericalValue=3)]), OutlierDetectionMethod(id=_:N7fe9d2de7fc843ceaa88c8934e3fed68, name=normalized median test, hasParameter=NumericalVariable(id=_:N34a8a6bfbc0b4d9091720b6a19f6a259, label=threshold, hasNumericalValue=3.0)), OutlierDetectionMethod(id=_:N9c0574b620ad43d2b426dc136bdd6f95, name=dynamic mean test, hasParameter=[NumericalVariable(id=_:Ncb90bc91512941e5a060245434f88af8, label=mean, hasNumericalValue=2.0), NumericalVariable(id=_:Ndd98d6e8fa344d37b25fc4661376c5cb, label=var, hasNumericalValue=1.0)])])

In [17]:
data_smoothing = m4i.Method(
    name='Low-pass filtering',
    description='applies a low-pass filtering on the data using a Gaussian weighted kernel of specified width to reduce spurious noise.',
    parameter=m4i.NumericalVariable(label='kernel', value=2.0)
)

In [18]:
post = pivmeta.PIVPostProcessing(
    label='Post processing',
    realizesMethod=data_smoothing
)
post.model_dump_jsonld()         

'{\n    "@context": {\n        "owl": "http://www.w3.org/2002/07/owl#",\n        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",\n        "m4i": "http://w3id.org/nfdi4ing/metadata4ing#",\n        "schema": "https://schema.org/",\n        "obo": "http://purl.obolibrary.org/obo/",\n        "pivmeta": "https://matthiasprobst.github.io/pivmeta#"\n    },\n    "@type": "pivmeta:PIVProcessingStep",\n    "rdfs:label": "Post processing",\n    "m4i:realizesMethod": {\n        "@type": "m4i:Method",\n        "schema:description": "applies a low-pass filtering on the data using a Gaussian weighted kernel of specified width to reduce spurious noise.",\n        "m4i:hasParameter": {\n            "@type": "m4i:NumericalVariable",\n            "rdfs:label": "kernel",\n            "m4i:hasNumericalValue": 2.0,\n            "@id": "_:Nac7df88b1dc441f6b2e96dfb83ac2868"\n        },\n        "name": "Low-pass filtering",\n        "@id": "_:Na9da580c2f334d37bcac0c3dffa549f8"\n    },\n    "@id": "_:Ndf1ca4a

## 3. Creating the full Meta document (connect information)

We created three processing steps:
1. pre (takes raw images)
2. proc
3. post (outputs result data)


In [19]:
post.hasOutput = ds

## 4. dump PIV run to JSON-LD

In [20]:
piv = m4i.ProcessingStep(
    label='PIV Run',
    startsWith=pre,
    endsWith=post
)
# proc.part_of = piv
pre.precedes = proc
proc.precedes = post

# all processing steps were employed by software pivview:
pre.has_employed_tool = piv_software
proc.has_employed_tool = piv_software
post.has_employed_tool = piv_software

In [21]:
print(pre.model_dump_jsonld())

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "m4i": "http://w3id.org/nfdi4ing/metadata4ing#",
        "schema": "https://schema.org/",
        "obo": "http://purl.obolibrary.org/obo/",
        "pivmeta": "https://matthiasprobst.github.io/pivmeta#",
        "ssno": "https://matthiasprobst.github.io/ssno#",
        "skos": "http://www.w3.org/2004/02/skos/core#",
        "dcat": "http://www.w3.org/ns/dcat#",
        "dcterms": "http://purl.org/dc/terms/",
        "prov": "http://www.w3.org/ns/prov#"
    },
    "@type": "pivmeta:PIVPostProcessing",
    "m4i:realizesMethod": [
        {
            "@type": "m4i:Method",
            "schema:description": "Rotates the input image by 180 deg",
            "m4i:hasParameter": {
                "@type": "m4i:NumericalVariable",
                "m4i:hasNumericalValue": 180,
                "m4i:hasUnit": "http://qudt.org/vocab/unit/DEG",
                "m4i

In [22]:
with open('piv_process.json', 'w') as f:
    f.write(proc.model_dump_jsonld())

In [23]:
print(piv.model_dump_jsonld())

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "m4i": "http://w3id.org/nfdi4ing/metadata4ing#",
        "schema": "https://schema.org/",
        "obo": "http://purl.obolibrary.org/obo/",
        "pivmeta": "https://matthiasprobst.github.io/pivmeta#",
        "ssno": "https://matthiasprobst.github.io/ssno#",
        "skos": "http://www.w3.org/2004/02/skos/core#",
        "dcat": "http://www.w3.org/ns/dcat#",
        "dcterms": "http://purl.org/dc/terms/",
        "prov": "http://www.w3.org/ns/prov#"
    },
    "@type": "m4i:ProcessingStep",
    "rdfs:label": "PIV Run",
    "startsWith": {
        "@type": "pivmeta:PIVPostProcessing",
        "m4i:realizesMethod": [
            {
                "@type": "m4i:Method",
                "schema:description": "Rotates the input image by 180 deg",
                "m4i:hasParameter": {
                    "@type": "m4i:NumericalVariable",
                   